In [ ]:
import geopandas as gpd
import rasterio as rio

import rasterstats

import matplotlib.pyplot as plt
from rasterio.plot import show

from os.path import join, basename
from glob import glob

In [ ]:
ROOT_DIR = '/media/sf_JD/DP'
stats = ['mean', 'median', 'std']

## Testing raster statistics

In [ ]:
def compute_zonal_stats_one_orbit(raster_dir, stats_gdf, stats=['mean']):
    # Get relative orbit and raster filenames
    ro = int(basename(raster_dir))
    rasters_list = glob('coh_*.tif', root_dir=raster_dir)
    
    print(f'Processing {len(rasters_list)} rasters:')
    # loop through all coherence rasters
    for raster in rasters_list:
        print(raster)
        
        # Read raster array and transform
        raster_path = join(raster_dir, raster)
        with rio.open(raster_path, mode='r', count=2) as src:
            arr = src.read()
            affine = src.transform
        
        # compute zonal statistics for individual polarizations
        for idx_band, pol in enumerate(('VH', 'VV')):
            prefix = f'{ro:03}_{pol}_{raster[4:8]}_{raster[8:12]}-{raster[17:21]}_'

            stats_list = rasterstats.zonal_stats(stats_gdf, arr[idx_band,:,:],
                                                 affine=affine, geojson_out=True,
                                                 prefix=prefix, stats=stats, nodata=-1)
            stats_gdf = gpd.GeoDataFrame.from_features(stats_list, crs=stats_gdf.crs)

    print('Zonal stats successfuly computed.')
    return stats_gdf

In [ ]:
def compute_zonal_stats_all_orbits(root_dir, year, stats):
    
    in_vector = join(root_dir, f'reference/{year}/vyjezdy_{year}_4326_singlepart.gpkg')
    out_vector = join(root_dir, f'reference/{year}/vyjezdy_{year}_zonal_stats.gpkg')
    raster_dir = join(root_dir, f's1_coherence/{year}')
    
    gdf_vector = gpd.read_file(in_vector)
    
    for ro_dir in glob(f'{raster_dir}/*'):
        print(ro_dir)
        gdf_vector = compute_zonal_stats_one_orbit(ro_dir, gdf_vector, stats)
    
    gdf_vector.to_file(out_vector)
    return gdf_vector

In [ ]:
def zonal_stats_all(root_dir, years=(2021,), stats=['mean']):
    for year in years:
        compute_zonal_stats_all_orbits(root_dir, year, stats)

In [ ]:
zonal_stats_all(ROOT_DIR, years=(2021,), stats=stats)

In [ ]:
ro = 22
f'{ro:03}'

In [ ]:
ro = 22
pol = 'VH'
raster = 'coh_20210517_20210523.tif'

f'{ro:03}_{pol}_{raster[4:8]}_{raster[8:12]}-{raster[17:21]}_'